# 고객요청의도 분석하기 (하수도 관련 문의 - 요금, 검침)
## 작성자 : AISchool ( http://aischool.ai/%ec%98%a8%eb%9d%bc%ec%9d%b8-%ea%b0%95%ec%9d%98-%ec%b9%b4%ed%85%8c%ea%b3%a0%eb%a6%ac/ )
## 민원(콜센터) 질의-응답 데이터 다운받기 : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=98

# LangChain 라이브러리 설치

In [ ]:
!pip install langchain openai chromadb tiktoken pypdf unstructured sentence-transformers langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.2/974.2 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd

# 데이터 읽어오기

In [ ]:
json_file_path = '/content/민원(콜센터) 질의응답_다산콜센터_생활하수도 관련 문의_Training.json'
df = pd.read_json(json_file_path)
df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
0,다산콜센터,생활하수도 관련 문의,B2355,고객,1,계량기고장,,Q,수도 계량기가 고장난 것 같습니다.,,,,"계량기, 고장",계량기/수도계량기/ 고장/고장,고장
1,다산콜센터,생활하수도 관련 문의,B2355,상담사,2,,계량기고장,Q,,증상이 어떻게 됩니까?,,,증상,증상/증상,증상
2,다산콜센터,생활하수도 관련 문의,B2355,고객,3,계량기고장,,A,,,침이 회전을 안합니다.,,"침, 회전",침/수도계량기/ 회전/수도계량기,"회전,수도계량기"
3,다산콜센터,생활하수도 관련 문의,B2355,상담사,4,,계량기고장,A,,,,고장 발생 시 교체를 위하여 민원 신고 및 자체 검침을 통해 교체해주셔야 합니다.,"고장, 교체, 민원 신고, 자체 검침",고장/고장/ 교체/행정서비스/ 민원 신고/민원/ 자체검침/검침유형,"교체,검침유형"
4,다산콜센터,생활하수도 관련 문의,B2355,고객,5,계량기고장,,Q,민원 신고는 어디서 합니까?,,,,민원 신고,민원 신고/민원,"민원신고,민원"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40612,다산콜센터,생활하수도 관련 문의,B36388,상담사,17,,요금납부,Q,,추가로 궁금하신점 있으십니까?,,,,,
40613,다산콜센터,생활하수도 관련 문의,B36388,고객,18,요금납부,,A,,,인터넷납부로 진행할게요.,,인터넷납부,인터넷납부/요금납부,"인터넷납부,요금납부"
40614,다산콜센터,생활하수도 관련 문의,B36388,상담사,19,,요금납부,A,,,,서울시ETAX시스템(http://etax.seoul.go.kr) 또는 각 은행 인터...,,,
40615,다산콜센터,생활하수도 관련 문의,B36388,상담사,20,,요금납부,Q,,추가로 궁금하신점 있으십니까?,,,,,


In [ ]:
len(df)

40617

In [ ]:
df.columns

Index(['도메인', '카테고리', '대화셋일련번호', '화자', '문장번호', '고객의도', '상담사의도', 'QA',
       '고객질문(요청)', '상담사질문(요청)', '고객답변', '상담사답변', '개체명 ', '용어사전', '지식베이스'],
      dtype='object')

# 각 column별 unique한 값 확인하기

In [ ]:
도메인_리스트 = df['도메인'].unique()
for idx, 도메인 in enumerate(도메인_리스트):
  print(idx, 도메인)

0 다산콜센터


In [ ]:
카테고리_리스트 = df['카테고리'].unique()
for idx, 카테고리 in enumerate(카테고리_리스트):
  print(idx, 카테고리)

0 생활하수도 관련 문의


In [ ]:
대화셋일련번호_리스트 = df['대화셋일련번호'].unique()
for idx, 대화셋일련번호 in enumerate(대화셋일련번호_리스트):
  print(idx, 대화셋일련번호)

0 B2355
1 B2356
2 B2357
3 B2358
4 B2359
5 B2360
6 B2361
7 B2362
8 B2363
9 B2364
10 B2365
11 B2366
12 B2367
13 B2368
14 B2369
15 B2370
16 B2371
17 B2372
18 B2373
19 B2374
20 B2375
21 B2376
22 B2377
23 B2378
24 B2379
25 B2380
26 B2381
27 B2382
28 B2383
29 B2384
30 B2385
31 B2386
32 B2387
33 B2388
34 B2389
35 B2390
36 B2391
37 B2392
38 B2393
39 B2394
40 B2395
41 B2396
42 B2397
43 B2398
44 B2399
45 B2400
46 B2401
47 B2402
48 B2403
49 B2404
50 B2405
51 B2406
52 B2407
53 B2408
54 B2409
55 B2410
56 B2411
57 B2412
58 B2413
59 B2414
60 B2415
61 B2416
62 B2417
63 B2418
64 B2419
65 B2420
66 B2421
67 B2422
68 B2423
69 B2424
70 B2513
71 B2514
72 B2515
73 B2516
74 B2517
75 B2518
76 B2519
77 B2520
78 B2521
79 B2522
80 B2523
81 B2524
82 B2525
83 B2526
84 B2527
85 B2528
86 B2529
87 B2530
88 B2531
89 B2641
90 B2655
91 B2712
92 B2713
93 B2714
94 B2715
95 B2716
96 B2717
97 B2718
98 B2719
99 B2720
100 B2721
101 B2722
102 B2723
103 B2724
104 B2725
105 B2726
106 B2727
107 B2728
108 B2729
109 B2730
110 B2731


In [ ]:
화자_리스트 = df['화자'].unique()
for idx, 화자 in enumerate(화자_리스트):
  print(idx, 화자)

0 고객
1 상담사


In [ ]:
문장번호_리스트 = df['문장번호'].unique()
for idx, 문장번호 in enumerate(문장번호_리스트):
  print(idx, 문장번호)

0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
21 22
22 23
23 24
24 25
25 26
26 27
27 28
28 29
29 30
30 31


In [ ]:
고객의도_리스트 = df['고객의도'].unique()
for idx, 고객의도 in enumerate(고객의도_리스트):
  print(idx, 고객의도)

0 계량기고장
1 
2 수도요금조회
3 수도요금납부
4 요금납부방법
5 정산요금확인
6 이사정산문의
7 구제방법문의
8 누수구분
9 상하수도정산
10 수질정보
11 계량기교체
12 명의변경
13 전자고지신청
14 요금확인방법
15 자동납부신청
16 서비스해지
17 보증금
18 밸브설치
19 수질검사신청
20 요금연체
21 다자녀혜택
22 단기급수신청
23 내역확인
24 납부방법변경
25 업종변경
26 납부내역확인
27 요금납부
28 요금정산
29 불편신고
30 청구서문의
31 수도요금체납
32 사용기간문의
33 명의문의
34 월별내역확인
35 납입증명서
36 이체확인
37 납입액확인
38 자동이체문의
39 수리접수
40 고지서재발급
41 요금체납
42 서비스신청
43 요금이체
44 요금확인
45 고지서분실
46 수도세 
47 하수도 악취 신고
48 수도요금 할인
49 수도계량기 설치
50 자가 수도검침
51 수도요금 확인
52 상수도 누진제
53 수도요금 기록
54 파손 배수관 신고
55 녹물 신고
56 수도관 교체
57 배관 질문
58 수돗물 부유물
59 자동이체 해지 
60 수돗물 악취
61 아리수 품질 확인 신청
62 납입방법문의
63 수수료문의
64 연체료문의
65 납부방법
66 요금문의
67 자동납부
68 수도요금
69 고지서 문의
70 이사 관련
71 건물철거
72 누수
73 민원
74 이사
75 싱크대문의
76 화장실변기
77 세탁실배관
78 자동이체
79 세대분리
80 요금관련
81 요금감면
82 수도검침
83 홈페이지
84 명의자변경
85 자동이체신청
86 계량기
87 단수
88 욕실 물 역류
89 아파트누수문의
90 우수관 배수
91 하수도배관역류
92 하수도배관시설
93 수도검침문의
94 상수도문의
95 요금할인 
96 고지서
97 고시서
98 수도요금문의
99 이사요금조회
100 계량기 고장
101 상수도요금
102 누수문의
103 고지서송달
104 안심확인제
105 급수중지
106 계량기검정
107 수도배관교체
108 수도배관교체비용
1

In [ ]:
상담사의도_리스트 = df['상담사의도'].unique()
for idx, 상담사의도 in enumerate(상담사의도_리스트):
  print(idx, 상담사의도)

0 
1 계량기고장
2 수도요금조회
3 수도요금납부
4 요금납부방법
5 정산요금확인
6 이사정산문의
7 구제방법문의
8 누수구분
9 상하수도정산
10 수질정보
11 계량기교체
12 명의변경
13 전자고지신청
14 요금확인방법
15 자동납부신청
16 서비스해지
17 보증금
18 밸브설치
19 수질검사신청
20 요금연체
21 다자녀혜택
22 단기급수신청
23 내역확인
24 납부방법변경
25 업종변경
26 납부내역확인
27 요금납부
28 요금정산
29 불편신고
30 청구서문의
31 수도요금체납
32 사용기간문의
33 명의문의
34 월별내역확인
35 납입증명서
36 이체확인
37 납입액확인
38 자동이체문의
39 수리접수
40 고지서재발급
41 요금체납
42 서비스신청
43 요금이체
44 요금확인
45 고지서분실
46 수도세 
47 하수도 악취 신고
48 수도요금 할인
49 수도계량기 설치
50 자가 수도검침
51 수도요금 확인
52 상수도 누진제
53 수도요금 기록
54 파손 배수관 신고
55 녹물 신고
56 수도관 교체
57 배관 질문
58 수돗물 부유물
59 자동이체 해지 
60 수돗물 악취
61 아리수 품질 확인 신청
62 납입방법문의
63 수수료문의
64 연체료문의
65 납부방법
66 요금문의
67 자동납부
68 수도요금
69 고지서 문의
70 이사 관련
71 건물철거
72 누수
73 민원
74 이사
75 싱크대문의
76 화장실변기
77 세탁실배관
78 자동이체
79 세대분리
80 요금관련
81 요금감면
82 수도검침
83 홈페이지
84 명의자변경
85 자동이체신청
86 계량기
87 단수
88 욕실 물 역류
89 아파트누수문의
90 우수관 배수
91 하수도배관역류
92 하수도배관시설
93 수도검침문의
94 상수도문의
95 요금할인 
96 고지서
97 수도요금문의
98 이사요금조회
99 계량기 고장
100 상수도요금
101 누수문의
102 고지서송달
103 안심확인제
104 급수중지
105 계량기검정
106 동파
107 수도배관교체
108 수도배관교체비용
10

In [ ]:
QA_리스트 = df['QA'].unique()
for idx, QA in enumerate(QA_리스트):
  print(idx, QA)

0 Q
1 A


In [ ]:
고객질문_요청_리스트 = df['고객질문(요청)'].unique()
for idx, 고객질문_요청 in enumerate(고객질문_요청_리스트):
  print(idx, 고객질문_요청)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
4319 90일이내에 다시 전화드리면 되네요?
4320 그럼 따로 신청안해도 되겠네요?
4321 이번달 고지서에 검침상태가 인정조정이라고 써있던데 이거 무슨 뜻이예요?
4322 그게 먼데요?
4323 그럼 뭔 기준으로 요금을 부과하나요?
4324 언제 수도검침 하러 오는지 알수 없어요?
4325 그럼 매달 인터넷 들가서 확인해야되는거예요?
4326 아래층에는 수압이 세고 물 잘나오던데 위로는 물이 안올라와요.
4327 그럼 신고접수 해주세요
4328 아 그래요. 언제쯤 방문해주실수있나요?
4329 아~ 그러니까 건물차체내의 문제라면 수도사업소에서 해줄수있는 부분이 없다는 얘기시죠?
4330 수도계량기 교체할라면 어떻게 해야되나요?
4331 그럼 저도 해당되겠네요
4332 교체할라면 어떻게 해야되나요?
4333 아 그래요. 전화번호좀 알려주세요
4334 유효기간은 어떻게 알수 있는데요?
4335 그럼 우리집은 교체하고 나면 6년 후에 다시 바꿔주겠네요
4336 그리고 자동납부 신청하고 싶은데요?
4337 여기서는 안되요?
4338 아 번거롭네 저 혹시 꼭 자동이체만 되나요?
4339 안녕하세요 상담원 OOO 입니다. 무엇을 도와드릴까요?
4340 요즘 수돗물에 문제가 많아서 찜찜해서 그러는데요 소독약 냄새나느거 괜찮은거 맞죠?
4341 물을 틀었는데 수돗물이 하얗게(우유빛) 보이는 이유가 뭔가요?
4342 약품처리되서 보이는게 아니란 말이죠?
4343 그럼 이렇게 하얗게 보이는건 언제쯤 없어지나요? 계속이러는건 아니죠?
4344 수돗물에서 붉은 녹물이 나오는데 어떻게 해야 하나요?
4345 그러니까요 어떻게 조치를 취해주셔야죠? 그냥 마실순 없자나요?
4346 녹물이 계속 나올까요?
4347 안녕하세요 문의좀 드리겠습니다.
4348 이상하게 제가 저번달하고 물을 비슷하게 쓴거 같은데 왜 요금이 많이 나왔을까요?
4349 그렇게 말씀해주시면 잘 모르겠구요 몇프로나 오른건가요?
4350 수도요금 이 어

In [ ]:
상담사질문_요청_리스트 = df['상담사질문(요청)'].unique()
for idx, 상담사질문_요청 in enumerate(상담사질문_요청_리스트):
  print(idx, 상담사질문_요청)

0 
1 증상이 어떻게 됩니까?
2 교체 신청하시겠습니까?
3 성함과 주소를 말씀해주시겠습니까?
4 더 궁금한 사항 있으십니까?
5 네 무엇을 도와드립니까?
6 고객번호 알고계십니까?
7 지역이 어디십니까?
8 네 무엇을 도와드릴까요?
9 어떤 요금 말씀이십니까?
10 자동납부를 계좌로 신청하십니까, 카드로 신청하십니까?
11 네 다른 문의사항 있으십니까?
12 자동납부 신청 방법은 계좌, 카드에 따라 다릅니다. 어떤 방식으로 안내해드립니까?
13 어떤 은행 이용하십니까?
14 네. 무엇을 도와드릴까요?
15 사용기간이 어떻게 되십니까?
16 다른 궁금하신 사항 있으십니까?
17 이사 가시는겁니까?
18 네. 더 궁금한 사항 있으십니까?
19 현재 납부 방법이 어떻게 되십니까?
20 경기도 상하수도 고객센터 번호 알려드릴까요?
21 수도 계량기가 고장났습니까?
22 고장사유가 어떻게 되십니까?
23 네 지역이 어디십니까?
24 지역구까지 말씀 해주시겠습니까?
25 다른 문의사항 있으십니까?
26 해지하지는 이유가 있으십니까?
27 혹시 법인이십니까?
28 임시급수 신청하시는 거 맞으십니까?
29 당월요금 말씀이십니까, 기존 납부내역 말씀이십니까?
30 네. 고객번호가 어떻게 되십니까?
31 언제부터 언제까지 조회해드립니까?
32 혹시 누수확인해보셨습니까?
33 업종변경 관련 신청 안내해 드립니까?
34 지역이 어떻게 되십니까?
35 교체 지역이 서울시 맞으십니까?
36 고객번호 말씀해주십시오.
37 고지서 명의는 어느 분 성함입니까?
38 감사합니다. 다른 문의사항 있으십니까?
39 네. 무엇을 도와드립니까?
40 고지서 상에 관리 번호 확인 알려주실 수 있습니까?
41 고지서에 납부자명이 어떻게 되어있습니까?
42 어떤 방식으로 납부하시겠습니까?
43 이미 이사를 하신겁니까?
44 아 그러신가요? 지침번호가 어떻게 되십니까
45 정산하시는 지역구가 어떻게 되십니까?
46 관리번호가 어떻게 됩니까?
47 명의자분 성함이 어떻게 됩니까?
48 사시는 지역이 어디

In [ ]:
고객답변_리스트 = df['고객답변'].unique()
for idx, 고객답변 in enumerate(고객답변_리스트):
  print(idx, 고객답변)

0 
1 침이 회전을 안합니다.
2 네. 신청하겠습니다
3 OOO, 서울시 강남구 OOO입니다.
4 OOO, OOO-OOOO-OOOO입니다.
5 감사합니다.
6 네 감사합니다.
7 아니요. 모릅니다. 
8 서대문구입니다.
9 네 감사합니다. 
10 네 알겠습니다. 
11 수도요금입니다. 
12 자동 납부로 하겠습니다.
13 계좌로 하겠습니다.
14 전화신청으로 하겠습니다. 
15 그럼 자동납부로 신청하겠습니다. 
16 네 알겠습니다.
17 신한은행입니다.
18 감사합니다. 
19 이번에 요금이 34,640원이 나왔습니다. 
20 44일입니다. 
21 알겠습니다. 
22 이 집에서는 이사가고 다른 집에는 전입할때 말입니다. 
23 알겠습니다
24 네 알려주셔서 감사합니다. 
25 그렇군요. 알겠습니다. 감사합니다. 
26 알겠습니다.
27 아뇨 없습니다
28 자동이체로 결제하고 있습니다. 
29 네. 알려주세요.
30 마포구입니다. 
31 네. 작동이 잘 안되는거 같습니다. 
32 유리부쪽이 파손되어서 계량기 확인이 안됩니다. 
33 알려주셔서 감사합니다. 
34 서울시입니다. 
35 강북구 입니다. 
36 그렇군요. 알겠습니다. 
37 아니요. 없습니다. 
38 네 알겠습니다. 변경하면 신고하겠습니다. 
39 아뇨 없습니다.
40 네 알고있습니다. 
41 이사로 인해서 해지하려고 합니다. 
42 알겠습니다. 감사합니다. 
43 개인입니다. 
44 그렇군요 알겠습니다. 
45 하나만 더 여쭙겠습니다. 
46 5천 제곱미터입니다. 
47 영등포구입니다.
48 아 그럼 얼른 알아보고 바로 신청하겠습니다.
49 네 맞습니다. 가설건축물에 임시 급수 신청하려고 합니다. 
50 네 알겠습니다. 그럼 허가 받고 신청하겠습니다. 
51 하나만 더 여쭙겠습니다 
52 기존 납부 내역입니다.
53 OOO-OOOO-OOO입니다. 
54 1월부터 8월까지 해주십시오. 
55 그럼 3월부터 8월까지 조회해주십시오. 
56 아뇨 확인 안해봤습니다. 
57 네 감사합니다. 하나 더 물어

In [ ]:
상담사답변_리스트 = df['상담사답변'].unique()
for idx, 상담사답변 in enumerate(상담사답변_리스트):
  print(idx, 상담사답변)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
4000 하수가 많이 모이는 지점마다 총인시설을 배치하고 있어서 괜찮습니다.
4001 하수처리시설을 관리하는데 사용하는 비용이라고 이해하시면 편하실 것 같습니다.
4002 재설립 같은 경우엔 시 예산도 들어가지만 구경별이용금에도 포함되어있습니다.
4003 걱정해주셔서 감사합니다!
4004 북한강처럼 큰 강에서 냄새날 이유가 없는데 이상하네요. 수질연구원분들 보내서 확인해보겠습니다.
4005 냄새나는데는 많은 요인이 있을수도 있는데 불순물이 쌓였다던지 폐수가 흘러들었다던지가 원인이 될수 있습니다.
4006 아마 농장 한 곳에서 오물을 유기한다고 강 전체가 악취가 나진 않을 것 같은데 장기적으로 계속 그래왔으면 원인이 될 것 같기도 합니다.
4007 그 근처에 농장이 1개로 있는걸로 확인되서 가서 확인해보겠습니다.
4008 지어지는 시설의 배수시설의 적정성, 오수, 우수 분리배관 여부 등을 확인하기 위한 서류입니다.
4009 배수설비 준공검사 신청서는 시청 홈페이지에 가셔도 되시고 그게 어려우시면 가까운 구,시청 방문하셔도 되세요.
4010 구비서류로는 준공후 설치 전, 중, 후 사진을 첨부해주시면 좋을 것 같습니다.
4011 매월 정기적으론 검사를 해도 하수도 계속 흘러들어오기 때문에 불가능할 것 같습니다.
4012 아 그런가요. 알겠습니다.
4013 네, 단체 방문하실 수 있게 버스 전용 주차장도 있습니다.
4014 아무래도 구경별이용금은 좀 더 체계화 된 것에 비해 물이용부담금은 객관적인 지표가 부족한 편입니다.
4015 보통 호수에서 냄새나는 원인이 많지 않은데 이상네요. 한번 확인해보겠습니다.
4016 공장 건에대해선 보통 한달 정도 소요될 수 있습니다.
4017 현재 북한강 수질은 2등급이고 매월 정기적으로 검사하고 있습니다.
4018 총 3단계로 되어있는데 단계가 올라갈수록 가격도 올라갑니다.
4019 예를 들어서 1단계 요금 곱하기 1단계 사용량 더하기 2단계 요금 곱하기 2단계 사용량

In [ ]:
개체명_리스트 = df['개체명 '].unique()
for idx, 개체명 in enumerate(개체명_리스트):
  print(idx, 개체명)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
8763 자가검침일, 어떻게
8764 어디, 기록
8765 아리수홈페이지, 입력, 가능
8766 어플
8767 모바일아리 앱
8768 근데, 자가검침, 점검
8769 확인, 점검
8770 착오, 입력, 방지, 1년, 1회, 확인, 검침, 실시
8771 문자, 점검, 안내, 일정, 통지
8772 깜박, 검침, 연락
8773 문자, 검침일, 안내, 2회, 이상, 미검침, 직원, 해지
8774 아래층, 수압, 물
8775 경우, 어떻게
8776 건물, 내부, 배관, 협소, 수압, 경우, 판단, 담당, 직원, 방문, 조사, 설명
8777 내부, 배관
8778 배부, 배관, 노후, 원인
8779 담당, 기사님, 방문, 접수
8780 자택, 주소
8781 00시00구1가0
8782 오늘, 방문, 가능
8783 오늘, 오후, 3시, 방문, 가능
8784 내부, 배관, 노후, 원인, 공사, 방법
8785 노호, 원인, 전문, 설비업체, 의뢰, 공사
8786 기사님, 확인
8787 노후, 배관, 교체, 비용, 지원
8788 기사님, 방문, 확인, 노후, 배관, 원인, 연락
8789 내부문제, 물
8790 몇일, 요새
8791 원인, 지대, 주변, 모두, 수압, 부족, 경우
8792 다른, 원인
8793 건물, 내부, 옥내, 배관, 문제, 경우
8794 자택주소, 기사님, 방문, 예약
8795 000구00
8796 30분, 기사님, 방문, 예정
8797 점검, 추후, 문의, 사항, 연락
8798 옆건물, 물
8799 30분
8800 건물, 내부, 관로, 벨브, 확인, 옥내, 누수, 발생, 확인, 부탁
8801 누수, 계량기, 확인
8802 수도공사, 공사비, 어떻게, 적용
8803 공동주택, 단독주택
8804 세대당, 부과
8805 건축, 면적, 일정, 금액, 인입급수관, 구경, 차등, 적용, 시설, 부담, 합계액, 부과
8806 급수, 공사, 소요, 기간, 얼마나
8807 신청일, 5일, 이내, 현장, 조사, 공

In [ ]:
용어사전_리스트 = df['용어사전'].unique()
for idx, 용어사전 in enumerate(용어사전_리스트):
  print(idx, 용어사전)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
8846 수도계량기/계량기, 헤어드라이기, 서서히/천히
8847 드라이기/가전, 고장
8848  물/수도, 서서히/천천히
8849 물/수도, 수도계량기/계량기, 고장
8850 교체, 비용, 고객님/고객, 부담
8851 무료/공짜, 교체, 기사/직업
8852 자연, 동파, 계량기, 망실, 경우/상황
8853 점검
8854 추후, 점검
8855 실시간/시간, 수질, 정보
8856 아리수, 모바일/어플리케이션/앱, 앱/어플리케이션, 확인
8857 핸드폰, 확인
8858 어디/위치서/위치, 조회
8859 강남구도곡1동/주소
8860 실시간/시간, 아리수, 수질, 마시기, 적합, 단계
8861 ph
8862 7
8863 강동구천동/주소
8864 8
8865 가장
8866 5
8867 몸, 물/수도, ph
8868 6.7, 7.6
8869 알칼리수/물, 이유
8870 면역/저항, 기능, 강화/정도, 항산화
8871 서울/지역/지명, 수돗물/수도, 아리수/물
8872 수도관, 정체, 물/수도, 관내, 이물/수도질,
8873 유리용기/용기, 사기, 보관/보존
8874 금속용기/용기
8875 금속용기/용기, 물/수도, 산화, 유리, 사기, 용기, 물/수도, 변화
8876 곳/위치, 30분/분
8877 왜/이유, 시간/시간
8878 염소, 냄새
8879 염새, 냄새
8880 녹차, 티백, 레몬, 조각, 염소, 냄새
8881 냉장고/생활가전
8882 수돗물/수도, 하얀빛
8883 소독약, 산소
8884 산소
8885 겨울철, 찬물/수도, 물/수도속, 산소, 현상
8886 찬물/수도, 산소
8887 수도관, 물/수도탱크, 관리
8888 녹물/수도, 조심/주의
8889 수돗물/수도, 청소, 세면대, 분홍색, 물/수도질, 배수관, 교체
8890 현상, 수돗물/수도
8891 타일, 광택제, 미량, 코발트, 성분, 분홍색, 발색
8892 수돗물/수도, 녹물/수도, 어떻게/방법
8893 수도사업소/공공기관, 검사, 수도관, 무

In [ ]:
지식베이스_리스트 = df['지식베이스'].unique()
for idx, 지식베이스 in enumerate(지식베이스_리스트):
  print(idx, 지식베이스)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
7039 확인,홈페이지
7040 정산요금,금액
7041 B,가능한목록
7042 F,회원가입시가능한목록
7043 명의변경,변경
7044 신청방법,방법
7045 등기권리,서류
7046 등기부등본,서류
7047 전자고지서,고지서
7048 청구고지서,고지서
7049 명의자,본인
7050 본인인증,인증
7051 상수도홈페이지,통신수단
7052 변경시점,시점
7053 알림서비스,서비스
7054 인증
7055 신청서류,서류
7056 납부방법,방법
7057 000,이름
7058 입력,등록
7059 888888.0,개인정보
7060 등록,신청
7061 전화,신청수단
7062 궁금,안내
7063 전용수납기,기계
7064 09:00-16:00,시간
7065 5일,날짜
7066 상하수도
7067 평소,평균
7068 사업소,등록
7069 누수요금적용신청서,서류
7070 가정집,수도세
7071 가정집,거주형태
7072 1개,세팅
7073 함께,통합
7074 가정용,징수
7075 문의,궁금
7076 운영,수도세
7077 등록,확인
7078 상가주택,건물형태
7079 용도,돈
7080 요금,용도
7081 어떻게,개선방법
7082 공사,장치
7083 자기부담,비용
7084 세입자,논의
7085 얼마,크기
7086 사용,확실
7087 도움,힘
7088 2배,배수
7089 7일,이용량
7090 00동,위치
7091 몇호,상세주소
7092 000호,상세주소
7093 수치,이용량
7094 누수,누출
7095 수도사용량,물이용량
7096 계량기검사,진단
7097 자가점검,자가검침
7098 자가검침,자가점검
7099 관할수도사업부,정부기관
7100 신청,서비스
7101 어플,앱
7102 아리수앱,어플
7103 아버지,사족
7104 음성,안내
7105 계량기점검,계량기검침
7106 날짜,언제
7107 확인,안내
7108 강남구서초동00빌라,위치
7109 2시,시간
7110 변경신청,상태
7111 3기,상태
7112 방문,메시지
7113 6

# '고객의도' 종류 확인하기

In [ ]:
고객의도_리스트 = df['고객의도'].unique()
for idx, 고객의도 in enumerate(고객의도_리스트):
  print(idx, 고객의도)

0 계량기고장
1 
2 수도요금조회
3 수도요금납부
4 요금납부방법
5 정산요금확인
6 이사정산문의
7 구제방법문의
8 누수구분
9 상하수도정산
10 수질정보
11 계량기교체
12 명의변경
13 전자고지신청
14 요금확인방법
15 자동납부신청
16 서비스해지
17 보증금
18 밸브설치
19 수질검사신청
20 요금연체
21 다자녀혜택
22 단기급수신청
23 내역확인
24 납부방법변경
25 업종변경
26 납부내역확인
27 요금납부
28 요금정산
29 불편신고
30 청구서문의
31 수도요금체납
32 사용기간문의
33 명의문의
34 월별내역확인
35 납입증명서
36 이체확인
37 납입액확인
38 자동이체문의
39 수리접수
40 고지서재발급
41 요금체납
42 서비스신청
43 요금이체
44 요금확인
45 고지서분실
46 수도세 
47 하수도 악취 신고
48 수도요금 할인
49 수도계량기 설치
50 자가 수도검침
51 수도요금 확인
52 상수도 누진제
53 수도요금 기록
54 파손 배수관 신고
55 녹물 신고
56 수도관 교체
57 배관 질문
58 수돗물 부유물
59 자동이체 해지 
60 수돗물 악취
61 아리수 품질 확인 신청
62 납입방법문의
63 수수료문의
64 연체료문의
65 납부방법
66 요금문의
67 자동납부
68 수도요금
69 고지서 문의
70 이사 관련
71 건물철거
72 누수
73 민원
74 이사
75 싱크대문의
76 화장실변기
77 세탁실배관
78 자동이체
79 세대분리
80 요금관련
81 요금감면
82 수도검침
83 홈페이지
84 명의자변경
85 자동이체신청
86 계량기
87 단수
88 욕실 물 역류
89 아파트누수문의
90 우수관 배수
91 하수도배관역류
92 하수도배관시설
93 수도검침문의
94 상수도문의
95 요금할인 
96 고지서
97 고시서
98 수도요금문의
99 이사요금조회
100 계량기 고장
101 상수도요금
102 누수문의
103 고지서송달
104 안심확인제
105 급수중지
106 계량기검정
107 수도배관교체
108 수도배관교체비용
1

# 고객의도가 '요금'과 관련된 데이터만 필터링하기

In [ ]:
요금_키워드_포함_고객의도_리스트 = [의도 for 의도 in 고객의도_리스트 if '요금' in 의도]
요금_키워드_포함_고객의도_리스트

['수도요금조회',
 '수도요금납부',
 '요금납부방법',
 '정산요금확인',
 '요금확인방법',
 '요금연체',
 '요금납부',
 '요금정산',
 '수도요금체납',
 '요금체납',
 '요금이체',
 '요금확인',
 '수도요금 할인',
 '수도요금 확인',
 '수도요금 기록',
 '요금문의',
 '수도요금',
 '요금관련',
 '요금감면',
 '요금할인 ',
 '수도요금문의',
 '이사요금조회',
 '상수도요금',
 '누수관련수도요금',
 '수도요금 영수증',
 '수도요금정산',
 '수도요금관련문의',
 '공동수도요금문의',
 '수도요금자동이체',
 '수도요금납부방법',
 '수도요금미납및단수',
 '누수로인한수도요금',
 '수도요금자동납부문의',
 '누수요금감면문의',
 '수도요금할인',
 '요금납부증명원',
 '수도요금 조회 문의',
 '체납요금',
 '수도요금 연체 문의',
 '요금감면제도',
 '수도요금납부연장',
 '수도요금감면',
 '수도요금납부 ',
 '수도요금 납부방법',
 '수도요금 계산방식',
 '상하수도요금',
 '요금처리문의',
 '수도요금 정보',
 '이사정산요금',
 '하수도 요금',
 '수도요금미납문의',
 '수도요금바로알림서비스',
 '기초생활수급자요금감면',
 '수도요금 미납',
 '수도요금알림',
 '소도요금문의',
 '요금계산',
 '누수요금',
 '과다요금문의',
 '다가구 수도요금',
 '요금고지서',
 '요금문의 ',
 '요금청구서 훼손',
 '요금납부 방법변경',
 '수도요금청구서',
 '수도요금계산',
 '요금납부변경',
 '요금자동이체',
 '다자녀요금감면',
 '수도요금고지서',
 '수도요금인상',
 '수도요금 자가검침',
 '상하수도 요금납부',
 '상하수도 요금 납부',
 '수도요금 납부 방법',
 '수도요금 자동이체',
 '수도요금 위택스',
 '수도요금 지로 고지서',
 '수도요금 조회',
 '수도요금 고지서',
 '수도 요금',
 '요금 납부 방법',
 '요금 지불방식',
 '수도요금 문의',
 '수도요금 

In [ ]:
filtered_df = df[(df['고객의도'] == '수도요금조회') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
20,다산콜센터,생활하수도 관련 문의,B2356,고객,1,수도요금조회,,Q,수도요금 문의하고 싶습니다.,,,,수도요금,수도요금/비용,"수도요금,비용"
1300,다산콜센터,생활하수도 관련 문의,B2420,고객,1,수도요금조회,,Q,수도요금 문의하고 싶어요.,,,,수도요금,수도요금/비용,"수도요금,비용"
3100,다산콜센터,생활하수도 관련 문의,B3449,고객,1,수도요금조회,,Q,수도요금 문의합니다.,,,,수도요금,수도요금/비용,"수도요금,비용"
9660,다산콜센터,생활하수도 관련 문의,B11363,고객,1,수도요금조회,,Q,안녕하세요. 수도요금 조회 가능한가요?,,,,요금,요금/비용,"요금,비용"
10020,다산콜센터,생활하수도 관련 문의,B15907,고객,1,수도요금조회,,Q,수도요금도 조회 가능한가요?,,,,수도요금조회,수도요금조회,수도요금조회
10040,다산콜센터,생활하수도 관련 문의,B15908,고객,1,수도요금조회,,Q,수도요금이 너무 많이 나왔습니다.,,,,수도요금,수도요금,수도요금
10260,다산콜센터,생활하수도 관련 문의,B16510,고객,1,수도요금조회,,Q,이번달 수도요금을 조회해보고 싶은데요.,,,,"수도요금, 조회","수도요금/비용, 조회/확인","조회,확인"
10440,다산콜센터,생활하수도 관련 문의,B16595,고객,1,수도요금조회,,Q,수도요금 좀 알고 싶어요.,,,,수도요금,수도요금/비용,"수도요금,비용"
11380,다산콜센터,생활하수도 관련 문의,B22248,고객,1,수도요금조회,,Q,수도요금을 조회하고 싶습니다.,,,,"수도요금, 조회","수도요금/요금, 조회",조회
11400,다산콜센터,생활하수도 관련 문의,B22249,고객,1,수도요금조회,,Q,수도요금 고지서가 안 왔어요. 수도요금 좀 알 수 있나요?,,,,"수도요금, 고지서","수도요금/요금, 고지서/서류","고지서,서류"


In [ ]:
filtered_df = df[(df['고객의도'] == '수도요금납부') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
40,다산콜센터,생활하수도 관련 문의,B2357,고객,1,수도요금납부,,Q,수도요금 문의합니다.,,,,수도요금,수도요금/비용,"수도요금,비용"
1320,다산콜센터,생활하수도 관련 문의,B2421,고객,1,수도요금납부,,Q,수도요금 문의하려는데요.,,,,수도요금,수도요금/비용,"수도요금,비용"
3120,다산콜센터,생활하수도 관련 문의,B3450,고객,1,수도요금납부,,Q,수도요금 문의합니다.,,,,수도요금,수도요금/비용,"수도요금,비용"
4020,다산콜센터,생활하수도 관련 문의,B3582,고객,1,수도요금납부,,Q,저기요 공과금을 편의점에서 납부도 할 수 있다고 하던데요?,,,,"공과금, 편의점, 납부",공과금/세금,"편의점,세금"
4100,다산콜센터,생활하수도 관련 문의,B3586,고객,1,수도요금납부,,Q,수고 많으세요 수도요금관해 궁금한게 있어서요.,,,,"수도요금, 궁금",수도요금/수도세,"궁금,수도세"
10460,다산콜센터,생활하수도 관련 문의,B16596,고객,1,수도요금납부,,Q,수도요금 문의할 수 있나요?,,,,수도요금,수도요금/비용,"수도요금,비용"
15620,다산콜센터,생활하수도 관련 문의,B23037,고객,1,수도요금납부,,Q,수도요금 납부방법을 변경하고 싶어요.,,,,"수도요금,납부방법,변경",수도요금/요금,"납부방법,요금"
15640,다산콜센터,생활하수도 관련 문의,B23038,고객,1,수도요금납부,,Q,수도요금 납부방법을 변경하고 싶어요.,,,,"수도요금,납부방법,변경",수도요금/요금,"납부방법,요금"
15660,다산콜센터,생활하수도 관련 문의,B23039,고객,1,수도요금납부,,Q,이번달 수도요금 납부방법을 변경하고 싶어요.,,,,"수도요금,납부방법,변경",수도요금/요금,"납부방법,요금"
15781,다산콜센터,생활하수도 관련 문의,B23045,고객,1,수도요금납부,,Q,이번달 수도요금 납부방법을 변경하고 싶어요.,,,,"수도요금,납부방법,변경",수도요금/요금,"납부방법,요금"


In [ ]:
filtered_df = df[(df['고객의도'] == '정산요금확인') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
140,다산콜센터,생활하수도 관련 문의,B2362,고객,1,정산요금확인,,Q,이사 정산 요금을 좀 확인하려고 합니다.,,,,정산 요금,정산 요금/요금,"정산요금,요금"
3220,다산콜센터,생활하수도 관련 문의,B3455,고객,1,정산요금확인,,Q,이사 정산 요금을 좀 확인하려고 합니다.,,,,정산 요금,정산 요금/요금,"정산요금,요금"
4340,다산콜센터,생활하수도 관련 문의,B3598,고객,1,정산요금확인,,Q,수고하십니다. 정산요금 확인 좀 하고 싶어서요,,,,"수고, 정산요금, 확인",수고/노고/ 확인/점검,"정산요금,점검"
10520,다산콜센터,생활하수도 관련 문의,B16599,고객,1,정산요금확인,,Q,이사 정산 요금 확인해보고 싶어요.,,,,정산 요금,정산 요금/요금,"정산요금,요금"
12918,다산콜센터,생활하수도 관련 문의,B22593,고객,1,정산요금확인,,Q,이사 정산 요금을 좀 확인하려는데요.,,,,정산 요금,정산 요금/요금,"정산요금,요금"
17342,다산콜센터,생활하수도 관련 문의,B23317,고객,1,정산요금확인,,Q,이사요금 정산을 했는데 확인 좀 하고 싶어서요.,,,,이사요금,이사요금/요금/비용/금액,"이사요금,금액"
19471,다산콜센터,생활하수도 관련 문의,B23450,고객,1,정산요금확인,,Q,이사합니다. 수도요금 정산하고 싶어요.,,,,"이사, 정산, 요금, 확인","정산/정리, 요금/요금","정산,요금"


# 고객의도가 '검침'과 관련된 데이터만 필터링하기

In [ ]:
검침_키워드_포함_고객의도_리스트 = [의도 for 의도 in 고객의도_리스트 if '검침' in 의도]
검침_키워드_포함_고객의도_리스트

['자가 수도검침',
 '수도검침',
 '수도검침문의',
 '검침',
 '수도자가검침',
 '검침원 정보',
 '검침문의',
 '자가검침',
 '검침신청',
 '자가검침 변경',
 '수도계량기 원격검침',
 '계량기검침이상, 누수',
 '수도요금 자가검침',
 '수도요금 검침 ',
 '계량기 검침',
 '수도계량기 검침 ']

In [ ]:
filtered_df = df[(df['고객의도'] == '자가 수도검침') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
1500,다산콜센터,생활하수도 관련 문의,B2518,고객,1,자가 수도검침,,Q,수도검침을 본인이하면 요금 감면이 되나요?,,,,"수도검침, 본인, 요금, 감면",요금/돈/ 감면/면제,"본인,면제"
1520,다산콜센터,생활하수도 관련 문의,B2519,고객,1,자가 수도검침,,Q,안녕하세요. 수도검침 문의드려요.,,,,수도검침,수도검침/검침,"수도검침,검침"
29574,다산콜센터,생활하수도 관련 문의,B25496,고객,1,자가 수도검침,,Q,수도검침을 본인이하면 요금 감면이 되는건가요?,,,,"수도검침, 본인, 요금, 감면","요금/돈, 감면/면제","본인,면제"
29594,다산콜센터,생활하수도 관련 문의,B25497,고객,1,자가 수도검침,,Q,안녕하세요. 수도검침 문의드립니다.,,,,수도검침,수도검침/검침,"수도검침,검침"
33757,다산콜센터,생활하수도 관련 문의,B26135,고객,1,자가 수도검침,,Q,수도검침을 본인이하면 요금 감면이 되는건가요?,,,,"수도검침, 본인, 요금, 감면","요금/돈, 감면/면제","본인,면제"
33777,다산콜센터,생활하수도 관련 문의,B26136,고객,1,자가 수도검침,,Q,안녕하세요. 수도검침 문의드립니다.,,,,수도검침,수도검침/검침,"수도검침,검침"


In [ ]:
filtered_df = df[(df['고객의도'] == '수도검침') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
2931,다산콜센터,생활하수도 관련 문의,B2939,고객,1,수도검침,,Q,저희집 계량기가 오래된것 같은데 교체는 어디에서 해주나요?,,,,"계량기,교체",,교체
9560,다산콜센터,생활하수도 관련 문의,B11358,고객,1,수도검침,,Q,안녕하세요. 저희집 수도 검침일은 언제인지 알수 있나요?,,,,"집, 검침일","집/거주지, 검침일/날짜","검침일,날짜"
10880,다산콜센터,생활하수도 관련 문의,B16741,고객,1,수도검침,,Q,우리집 수도 검침일이 궁금한데요?,,,,"집, 수도, 검침일","집/거주지, 검침일/날짜","수도,날짜"
11300,다산콜센터,생활하수도 관련 문의,B16762,고객,1,수도검침,,Q,우리 집 수도 검침 언제 하는지 궁금하네요?,,,,"집, 수도검침, 언제, 궁금","집/거주지, 수도검침/검침","수도검침,검침"
21487,다산콜센터,생활하수도 관련 문의,B23666,고객,1,수도검침,,Q,우리집 수도는 언제 검침하나요?,,,,"우리집, 수도, 언제, 검침","수도, 언제/시기, 검침/검침","수도,검침"
25971,다산콜센터,생활하수도 관련 문의,B24421,고객,1,수도검침,,Q,저희집 수도 검침일은 언제인가요?,,,,"집, 수도, 검침일","집/거주지, 검침일/날짜","수도,날짜"


In [ ]:
filtered_df = df[(df['고객의도'] == '수도검침문의') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
3880,다산콜센터,생활하수도 관련 문의,B3575,고객,1,수도검침문의,,Q,수고하십니다 저희집 수도검침이 잘못된것 같아서요,,,,"저희,집, 수도검침",저희/우리,"집,우리"
10180,다산콜센터,생활하수도 관련 문의,B16190,고객,1,수도검침문의,,Q,안녕하세요 저희집 수도검침이 잘못된것 같아요,,,,"저희,집, 수도검침",저희집-우리집 수도검침-수도,"집,저희집-우리집수도검침-수도"
16142,다산콜센터,생활하수도 관련 문의,B23255,고객,1,수도검침문의,,Q,저희집 수도 검침일은 언제인가요?,,,,"집, 수도, 검침일","집/거주지, 검침일/날짜","수도,날짜"
16162,다산콜센터,생활하수도 관련 문의,B23256,고객,1,수도검침문의,,Q,저희집 수도 검침일은 언제인가요?,,,,"집, 수도, 검침일","집/거주지, 검침일/날짜","수도,날짜"
16182,다산콜센터,생활하수도 관련 문의,B23257,고객,1,수도검침문의,,Q,수도 검침일 무느이드립니다,,,,"집, 수도, 검침일","집/거주지, 검침일/날짜","수도,날짜"


In [ ]:
filtered_df = df[(df['고객의도'] == '검침') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
5917,다산콜센터,생활하수도 관련 문의,B4000,고객,1,검침,,Q,수도요금 검침 언제 하나요?,,,,"집, 수도검침, 언제, 궁금",집/거주지/ 수도검침/검침,"수도검침,검침"
12540,다산콜센터,생활하수도 관련 문의,B22356,고객,1,검침,,Q,우리 집 수도 검침 언제 하는지 궁금해요.,,,,"집, 수도검침, 언제, 궁금","집/거주지, 수도검침/검침","수도검침,검침"
39430,다산콜센터,생활하수도 관련 문의,B35996,고객,1,검침,,Q,우리 집 수도 검침 언제 하는지 궁금해요.,,,,"집, 수도검침, 언제, 궁금","집/거주지, 수도검침/검침","수도검침,검침"


In [ ]:
filtered_df = df[(df['고객의도'] == '수도자가검침') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
6558,다산콜센터,생활하수도 관련 문의,B4035,고객,1,수도자가검침,,Q,수도도 자가검침이 가능한가요?,,,,"수도, 자가검침",수도/상수도/물/ 검침/검사,"자가검침,검사"
6578,다산콜센터,생활하수도 관련 문의,B4036,고객,1,수도자가검침,,Q,할인도 되나요?,,,,할인,할인/감면,"할인,감면"


In [ ]:
filtered_df = df[(df['고객의도'] == '검침원 정보') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
15118,다산콜센터,생활하수도 관련 문의,B22818,고객,1,검침원 정보,,A,,,네,,,,
38291,다산콜센터,생활하수도 관련 문의,B35895,고객,1,검침원 정보,,Q,여기가 해피라움 상가 거든요. 혹시 여기 수도검침하시는분 연락처 알 수 있을까요?,,,,"해피라움상가,수도검침,연락처",상가/검침원/수도검침/연락처,"수도검침,연락처"


In [ ]:
filtered_df = df[(df['고객의도'] == '검침문의') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
38433,다산콜센터,생활하수도 관련 문의,B35902,고객,1,검침문의,,Q,이번달 고지서에 검침상태가 인정조정이라고 써있던데 이거 무슨 뜻이예요?,,,,"고지서,검침상태,인정조정",수도요금/검침원/검침/인정조정/고지서,"검침상태,고지서"


# 사용자 질문으로부터 '고객요청의도' 추출하기

##  OpenAI API Key 설정

In [ ]:
OPENAI_KEY = "여러분의_OPENAI_API_KEY"

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=OPENAI_KEY)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# create_tagging_chain으로 고객 요청의도 추출하기

In [ ]:
from langchain.chains import create_tagging_chain

# 고객요청의도 추출을 위한 schema 설정
schema = {
    "properties": {
        "고객요청의도": {
            "type": "string",
            "enum": ['수도요금',
                     '검침'
            ]
        }
    }
}
chain = create_tagging_chain(schema, llm)

In [ ]:
chain_run_result = chain.run("수도요금 문의하고 싶습니다.")
chain_run_result

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'고객요청의도': '수도요금'}

In [ ]:
chain_run_result = chain.run("수도검침을 본인이하면 요금 감면이 되나요?")
chain_run_result

{'고객요청의도': '검침'}

In [ ]:
chain_run_result = chain.run("할인도 되나요?")
chain_run_result

{}